In [ ]:
!pip3 install onnxruntime

In [2]:
import pandas as pd
import numpy as np
import onnxruntime as rt


In [4]:
# preload model
respeck = rt.InferenceSession("model/respeck.onnx")
thingy = rt.InferenceSession("model/thingy.onnx")
input_name_respeck = respeck.get_inputs()[0].name
label_name_respeck = respeck.get_outputs()[0].name
input_name_thingy = thingy.get_inputs()[0].name
label_name_thingy = thingy.get_outputs()[0].name

class_labels = {
    'Desk work': 0,
    'Walking at normal speed': 1,
    'Standing': 2 ,
    'Sitting bent forward': 3,
    'Sitting': 4,
    'Sitting bent backward': 5,
    'Lying down right': 6,
    'Lying down left':7 ,
    'Lying down on back':8 ,
    'Lying down on stomach': 9, 
    'Movement': 10, 
    'Running': 11, 
    'Climbing stairs':12,
    'Descending stairs': 13
}


In [5]:
def rep_predict():
    resDf = pd.read_csv("cache/res.csv")
    res_columns_of_interest = [
        'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z'
    ]

    resFeatures = []
    
    for feature in res_columns_of_interest:
        data = np.array(resDf[feature].to_list())
        resFeatures.append(np.sum(data))
        resFeatures.append(np.median(data))
        resFeatures.append(np.mean(data))
        resFeatures.append(50)
        resFeatures.append(np.std(data))
        resFeatures.append(np.var(data))
        resFeatures.append(np.sqrt(np.mean(data**2)))
        resFeatures.append(max(data))
        resFeatures.append(max(map(abs, data)))
        resFeatures.append(min(data))
    resFeatures = np.array([resFeatures])
    resFeatures = resFeatures.astype(np.float32)
    pred_onx = respeck.run([label_name_respeck], {input_name_respeck: resFeatures})[0]
    return pred_onx
        

In [7]:
def thingy_predict():
    
    thiDf = pd.read_csv("cache/thi.csv")

    thi_columns_of_interest = [
        'accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x',
        'mag_y', 'mag_z'
    ]

    thiFeatures = []

    for feature in thi_columns_of_interest:
        data = np.array(thiDf[feature].to_list())
        thiFeatures.append(np.sum(data))
        thiFeatures.append(np.median(data))
        thiFeatures.append(np.mean(data))
        thiFeatures.append(50)
        thiFeatures.append(np.std(data))
        thiFeatures.append(np.var(data))
        thiFeatures.append(np.sqrt(np.mean(data**2)))
        thiFeatures.append(max(data))
        thiFeatures.append(max(map(abs, data)))
        thiFeatures.append(min(data))
    thiFeatures = np.array([thiFeatures])
    thiFeatures = thiFeatures.astype(np.float32)
    pred_onx = thingy.run([label_name_thingy], {input_name_thingy: thiFeatures})[0]
    return pred_onx
        

In [24]:
import json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)



In [26]:
numpyData = {"result": thingy_predict()}
encodedNumpyData = json.dumps(numpyData, cls=NumpyEncoder)

print(encodedNumpyData)

{"array": [10]}
